In [433]:
import os
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from scipy.io import loadmat
%matplotlib inline
sns.set()
sns.set_style('white')

In [434]:
data = loadmat('E:/Hamed/Amozesh/AI/Coursera-ML/16. Recommender Systems (Week 9)/ex8/ex8_movies')
r = data['R']
y = data['Y']

In [435]:
def average_rating(y, r, which_movie):
    return np.dot(y[which_movie, :], r[which_movie, :].T)

In [436]:
data = loadmat('E:/Hamed/Amozesh/AI/Coursera-ML/16. Recommender Systems (Week 9)/ex8/ex8_movieParams')
x = data['X']
theta = data['Theta']

In [437]:
num_users = 4
num_movies = 5
num_features = 3

x = x[:num_movies, :num_features]
theta = theta[:num_users, :num_features]
y = y[:num_movies, 0:num_users]
r = r[:num_movies, 0:num_users]

In [438]:
def flattenParams(myX, myTheta):
    return np.concatenate((myX.flatten(),myTheta.flatten()))

In [439]:
def reshapeParams(flattened_XandTheta, mynm, mynu, mynf):
    assert flattened_XandTheta.shape[0] == int(nm*nf+nu*nf)
    
    reX = flattened_XandTheta[:int(mynm*mynf)].reshape((mynm,mynf))
    reTheta = flattened_XandTheta[int(mynm*mynf):].reshape((mynu,mynf))
    
    return reX, reTheta

In [440]:
def collaborative_filtering_cost_function(theta, x, y, r, l):
    j = 1 / 2 * np.sum(np.square((np.dot(x, theta.T) - y) * r)) + l / 2 * (np.sum(np.square(theta)) + np.sum(np.square(x)))
    return j

In [441]:
collaborative_filtering_cost_function(theta, x, y, r, 1.5)

31.34405624427422

In [442]:
def collaborative_filtering_gradient(theta, x, y, r, l):
    Xgrad = np.dot((np.dot(x, theta.T) - y) * r, theta) + l * x
    Thetagrad = np.dot(((np.dot(x, theta.T) - y) * r).T, x) + l * theta
        
    return flattenParams(Xgrad, Thetagrad)